### Clear Sky

In [2]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import bisect
import datetime
from scipy import interpolate
from matplotlib.dates import DateFormatter

# User defined functions
from load_data_from_csv import *

#### Import  Data

In [3]:
day = '2018-10-12'   # select day of observation (camera 2, sw-vers. 3)
s_time = ' 09:00:00' # beginning of observation
e_time = ' 15:00:00' # end of observation

#### Weather station at Luzern Switzerland

In [4]:
luz_csv = r'irradiation_luz_2017_2018.csv'

df_luz = process_LUZ(luz_csv)
df_luz.set_index(df_luz.datetime, inplace=True)
df = df_luz['gre000z0']       # 10 min mean in W/m² )
#df.dtypes
luz_rad = df.loc[day+s_time:day+e_time]

#luz_rad = df[(df['datetime'] > day + start_time) & (df['datetime'] <= day + end_time)]

luz_rad.head(n=1)

datetime
2018-10-12 09:00:00+02:00    163
Freq: 10T, Name: gre000z0, dtype: object

#### Clear Sky solar irradiance from SODA using McClear model
Source: http://www.soda-pro.com/web-services/radiation/cams-mcclear

In [14]:
soda_csv = r'irradiation_soda_2017_2018_1min.csv'

df_soda = process_SODA(soda_csv)
mc_rad = df_soda['Clear sky GHI']/60 # in units of Wh/m² 
#mc_rad.head(n=1)

#### Solar measurements vs. McClear model

In [ ]:
fig = plt.figure(1, figsize=(10,4))
plt.plot(dateaxis, mc_rad, 'm', label='McClear model')
plt.plot(dateaxis, luz_rad, 'r', label='Solar radiation measurements')
plt.xlabel('Solar radiation [W/m²]', fontsize=12)
plt.legend(loc='upper left')
plt.grid(b=None, which='major', axis='both')
plt.title('2018-10-12')
